In [ ]:
!pip install cheshire_cat_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.0 MB/s eta 0:00:00


In [ ]:
import time
import json
import cheshire_cat_api as ccat

content = ""

def on_open():
    # This is triggered when the connection is opened
    print("Connection opened!")

def on_message(message: str):
    # This is triggered when a new message arrives
    # and grabs the message
    # print(message)
    global content

    response = json.loads(message)
    if response["type"] == "chat":
      content = response

def on_error(exception: Exception):
    # This is triggered when a WebSocket error is raised
    global content
    content = str(exception)

def on_close(status_code: int, message: str):
    # This is triggered when the connection is closed
    print(f"Connection closed!")



def connect(user_id):
  # Connection settings with default values
  config = ccat.Config(
      base_url="",
      port=31848,
      user_id=user_id,
      auth_key="",
      secure_connection=False
  )

  # Cat Client
  cat_client = ccat.CatClient(
      config=config,
      on_open=on_open,
      on_close=on_close,
      on_message=on_message,
      on_error=on_error
  )

  # Connect to the WebSocket API
  cat_client.connect_ws()

  while not cat_client.is_ws_connected:
      time.sleep(1)

  return cat_client

def disconnect(cat_client):
  cat_client.close()


def ask_gpt(cat_client, prompt):
  global content

  content = ""
  cat_client.send(message=prompt)

  while content == "":
    True

  return content


def prompt(user_id, prompt):
  client = connect(user_id)
  res = ask_gpt(client, prompt)
  disconnect(client)
  print(res)
  return res["content"], [( x["metadata"]["source"], x["score"], x["page_content"]) for x in res["why"]["memory"]["declarative"]]


def print_response(res):
  print("Output:\n")
  print(res[0])
  print("\n\nMemories:\n\n")
  for x in res[1]:
    print(f"{x[0]} - {x[1]}\n{x[2]}\n\n")


def clean_history(user):
  import requests

  url = ""

  payload = {}
  headers = {
    'Accept': 'application/json',
    'user_id': user
  }

  response = requests.request("DELETE", url, headers=headers, data=payload)
  print(response.text)

def get_history(user):
  import requests

  url = ""

  payload = {}
  headers = {
    'Accept': 'application/json',
    'user_id': user
  }

  response = requests.request("GET", url, headers=headers, data=payload)
  print(response.text)

def upload_memory(user, chunk_size, file_name, file_path):
  import http.client
  import mimetypes
  from codecs import encode
  import time

  conn = http.client.HTTPConnection("", 31848)
  dataList = []
  boundary = 'WebKitFormBoundary7MA4YWxkTrZu0gW'
  dataList.append(encode('--' + boundary))
  dataList.append(encode('Content-Disposition: form-data; name=file; filename={0}'.format(file_name)))

  fileType = mimetypes.guess_type(file_path)[0] or 'application/octet-stream'
  dataList.append(encode('Content-Type: {}'.format(fileType)))
  dataList.append(encode(''))

  with open(file_path, 'rb') as f:
    dataList.append(f.read())
  dataList.append(encode('--' + boundary))
  dataList.append(encode('Content-Disposition: form-data; name=chunk_size;'))

  dataList.append(encode('Content-Type: {}'.format('text/plain')))
  dataList.append(encode(''))

  dataList.append(encode(str(chunk_size)))
  dataList.append(encode('--' + boundary))
  dataList.append(encode('Content-Disposition: form-data; name=chunk_overlap;'))

  dataList.append(encode('Content-Type: {}'.format('text/plain')))
  dataList.append(encode(''))

  dataList.append(encode("100"))
  dataList.append(encode('--'+boundary+'--'))
  dataList.append(encode(''))
  body = b'\r\n'.join(dataList)
  payload = body
  headers = {
    'Content-type': 'multipart/form-data; boundary={}'.format(boundary),
    'Accept': 'application/json',
    'user_id': user
  }
  conn.request("POST", "/rabbithole/", payload, headers)
  res = conn.getresponse()
  data = res.read()
  print(data.decode("utf-8"))

def upload_all_memories_folder(user, chunk_size, folder):
  import os
  files = os.listdir(folder)

  for file_name in files:
    file_path = os.path.join(folder, file_name)
    upload_memory(user, chunk_size, file_name, file_path)
    time.sleep(1)



def summarise_in_folder(user, folder, target_folder):
  import os
  files = os.listdir(folder)

  for file_name in files:
    file_path = os.path.join(folder, file_name)
    prompt_summary_r = prompt_summary(file_path)
    clean_history(user)
    summary = prompt(user, prompt_summary_r)
    save_result(summary, f"{target_folder}/{file_name}")

In [ ]:
contract_loan_g = """

stipula Loan {
asset a
field field1, field2, t_0, t_1, t_1a, t_2, t_2a, field3, field4
init Q1

agreement (Lender, Borrower)(field1, field2, t_0, t_1, t_1a, t_2, t_2a){
	Lender, Borrower : field1, field2, t_0, t_1, t_1a, t_2, t_2a
} ==> @Q1

    @Q1 Borrower: f1()[] {
        "request" -> Lender;
        t_0 >> @Q2 {"end" -> Lender, Borrower} ==> @Qf
    } ==> @Q2

    @Q2 Lender: f2()[l] (l == field1){
        l -o Borrower;
        t_1 >> @Q3 {"payment_due" -> Borrower} ==> @Q4
        t_1a >> @Q4 {550 -> field3} ==> @Q4
        t_2 >> @Q5 {"payment_due" -> Borrower} ==> @Q6
        t_2a >> @Q6 {525 -> field3} ==> @Q6
    } ==> @Q3

    @Q4 Borrower: f3()[a] (a == 550){
        a -o Lender
        a -> field4;
        _
    } ==> @Q5

    @Q4 Lender: fn()[] (field3 > 0){
        "notice" -> Borrower;
        now + 48 >> @Q4 {"default" -> Borrower} ==> @Qd
    } ==> @Q4


    @Q6 Lender: fn()[] (field3 > 0){
        "notice" -> Borrower;
        now + 48 >> @Q6 {"default" -> Borrower} ==> @Qd
    } ==> @Q6


    @Q6 Borrower: f4()[a] (a == 525){
        a -o Lender;
        _
    } ==> @Qf


    @Qd Borrower: f5()[a] (a == field1 + (field1 * field2) - field4){
        a -o Lender;
        _
    } ==> @Qf

}
"""


contract_loan2_nat = """

This loan agreement dated June 1, 2022, by and between Lender Bank Co. (“Lender”) and
Borrower Corp. (Borrower), will set out the terms under which Lender will extend credit in the
principal amount of $1,000 to Borrower with an un-compounded interest rate of 5% per annum,
included in the specified payment structure.

1. The Loan
At the request of Borrower, Lender will advance $1,000 to Borrower
no later than June 2, 2022.

2. Repayment
Subject to the other terms of this agreement, Borrower will repay the loan in the following
payments:
(a) Payment 1, due June 1, 2023, in the amount of $550, represenng a payment of $500 as
half of the principal and interest in the amount of $50.
(b) Payment 2, due June 1, 2024, in the amount of $525, represenng a payment of $500 as
the remaining half of the principal and interest in the amount of $25.

3. Events of Default
The Borrower will be in default under this agreement upon the occurrence of any of the
following events or conditions, provided they shall remain uncured within a period of two days
after notice is given to Borrower by Lender of their occurrence (such an uncured event an “Event
of Default”):
(a) Borrower shall fail to make timely payment of any amount due to Lender hereunder;
A default will be cured by the Borrower (i) remedying the potential event of default and (ii)
giving effective notice of such remedy to the Lender.

4. Acceleration on Default
Upon the occurrence of an Event of Default all outstanding payments under this agreement will
become immediately due and payable, including both principal and interest amounts, without
further notice, presentment, or demand to the Borrower.

5. Courts and Litigation
Any legal action brought to enforce, interpret or otherwise deal with this agreement must be
brought in the state courts of the State of New York located in New York County, and each of the
parties agrees to the jurisdiction of such courts over both the parties themselves and over the
subject maer of such a proceeding, and waives any claim that such a court may be an
inconvenient forum.

6. Time of the Essence; No Pre-Payment
Timely performance is required for any action to be taken under this agreement, and, except as
may otherwise be specifically provided herein, failure to take such action on the day specified
will constitute a binding failure to take such action. Payments shall only be made on or after the
dates specified in Section 2 or on or after such other date as may be required under Section 6;
pre-payments made on earlier dates shall not be accepted.

"""


In [15]:
prompt_states_func = """
You will receive a contract written in the stipula programming language.
- identify all the functions that show the state @Q4 in their whole body;
- return a list of all functions found and print the referenced piece of code.

Follow the following structure:
- function name: piece of code;
"""

prompt_states_func2 = """
You will receive a list of functions written in the stipula programming language.
1. for each function, identify what party can invoke the function;
2. for each function, identify in what state the function can be invoked;
3. print the results
4. according to step 3, discard all the functions that cannot be called in state @Q4.
5. print the piece of code of the functions remaining from step 4.
"""


prompt_func_expl = """
You will receive a list of functions in the stipula programming language that are only callable from state @Q4.
Describe their runtime functioning according to the following structure.
- function: name the function - print the relevant piece of code
- state of invocation:
- pre-conditions for invocation: describe the preconditions - print the relevant piece of code
- operations triggered: describe one by one each operation and event described in the body of the function - print the relevant piece of code
- ending state: final state transition

Do not speculate on the legal meaning of the code.

"""


prompt_final = """
You have received the natural language contract and a the explanation of a function automating part of its lifecycle.
Identify, step by step, what each aspect of the function represent in the natural language contract.
Follow the structure:
- contract: article title related to the function;
  function: name the state of invocation - print the relevant piece of code
  function aspect: describe one function operation representing/implementing the contract;
  explanation: explain the match between contract and the aspect;
  contract text: "citation of the specific contract text relating to the aspect"

Some aspects of the function can have no direct correspondence with the contract.
Indicate where ther is no specifc correspondence.


"""

res9 = """
Based on the provided functions and their descriptions, here are the results:

1. Parties that can invoke the functions:
   - f2: Lender
   - f3: Borrower
   - fn (first occurrence): Lender

2. States in which the functions can be invoked:
   - f2: @Q2
   - f3: @Q4
   - fn (first occurrence): @Q4

3. Discarding functions that cannot be called in state @Q4:
   - Discarded f2 because it can only be invoked in state @Q2.

4. Functions remaining after discarding those not callable in state @Q4, along with their pieces of code:

- f3:
```
@Q4 Borrower: f3()[a] (a == 550){
    a -o Lender
    a -> field4;
    _
} ==> @Q5
```

- fn (first occurrence):
```
@Q4 Lender: fn()[] (field3 > 0){
    "notice" -> Borrower;
    now + 48 >> @Q4 {"default" -> Borrower} ==> @Qd
} ==> @Q4
```

"""

expl = """

 Based on the provided functions and their descriptions, here are the detailed runtime functioning of the functions callable from state @Q4:

### Function: f3
- **Code:**
```
'@Q4 Borrower: f3()[a] (a == 550){
    a -o Lender
    a -> field4;
    _
} ==> @Q5
```
- **State of Invocation:** @Q4
- **Pre-conditions for Invocation:** The asset `a` must be equal to 550.
- **Operations Triggered:**
  - The asset `a` is transferred to the Lender (`a -o Lender`).
  - The value of `a` is assigned to `field4` (`a -> field4`).
  - The contract transitions to state @Q5.
"""

# ### Function: fn (first occurrence)
# - **Code:**
# ```
# '@Q4 Lender: fn()[] (field3 > 0){
#     "notice" -> Borrower;
#     now + 48 >> @Q4 {"default" -> Borrower} ==> @Qd
# } ==> @Q4
# ```
# - **State of Invocation:** @Q4
# - **Pre-conditions for Invocation:** The value of `field3` must be greater than 0.
# - **Operations Triggered:**
#   - A "notice" is sent to the Borrower (`"notice" -> Borrower`).
#   - An event is scheduled to occur 48 time units from the current time. If the contract is still in state @Q4 when this time expires, a "default" message is sent to the Borrower, and the contract transitions to state @Qd (`now + 48 >> @Q4 {"default" -> Borrower} ==> @Qd`).
#   - The contract remains in state @Q4 after the function execution (`} ==> @Q4`).
# """


# Prompt NO Memoria

In [ ]:
upload_all_memories_folder("Cod_com2", 1000, "cod_com+funz")

# Experiments AI and Law

In [ ]:
clean_history("ciao")
res10 = prompt("ciao", f"Contract:\n\n{contract_loan2_nat} \n\n{prompt_con1}\n\nfunctions:\n\n{expl}")
print_response(res10)

{"deleted":true}
Connection opened!
Connection closed!
{'type': 'chat', 'user_id': 'ciao', 'content': '### Function: fn (first occurrence)\n- **contract: Events of Default**\n  - **function: fn()**\n  - **function aspect: "notice" -> Borrower**\n    - **explanation:** This operation represents the contract\'s requirement for the Lender to notify the Borrower upon the occurrence of an event that could lead to a default. This is a direct implementation of the contract\'s provision that the Borrower must be given notice of an event of default.\n    - **contract text:** "provided they shall remain uncured within a period of two days after notice is given to Borrower by Lender of their occurrence"\n- **function aspect: now + 48 >> @Q4 {"default" -> Borrower} ==> @Qd**\n  - **explanation:** This operation schedules a transition to a default state (@Qd) if the Borrower does not cure the default within 48 time units from the notice, mirroring the contract\'s allowance of a two-day period for t

In [ ]:
clean_history("ciao")
res10 = prompt("ciao", f"Contract:\n\n{contract_loan2_nat} \n\n{prompt_con1}\n\nfunctions:\n\n{expl}")
print_response(res10)

{"deleted":true}
Connection opened!
Connection closed!
{'type': 'chat', 'user_id': 'ciao', 'content': '### Contract: Repayment\n- **Function:** f3\n- **Function Aspect:** The asset `a` must be equal to 550.\n- **Explanation:** This aspect of the function corresponds to the first payment due under the contract, which is specified as $550 due on June 1, 2023. The function checks that the amount `a` being paid is exactly $550, aligning with the contract\'s stipulation for the first payment.\n- **Contract Text:** "Payment 1, due June 1, 2023, in the amount of $550, representing a payment of $500 as half of the principal and interest in the amount of $50."\n\n### Contract: Repayment\n- **Function:** f3\n- **Function Aspect:** The asset `a` is transferred to the Lender (`a -o Lender`).\n- **Explanation:** This operation represents the actual payment process where the specified amount (`a`) is transferred from the Borrower to the Lender, fulfilling the repayment obligation for the first insta

In [ ]:
clean_history("Cod_com2")
fun = prompt("Cod_com2", f"{prompt_func_expl}\n\n functions:\n\n{res9}")
print_response(fun)

{"deleted":true}
Connection opened!
Connection closed!
{'type': 'chat', 'user_id': 'Cod_com2', 'content': 'Based on the provided functions and their descriptions, here are the detailed runtime functioning of the functions callable from state @Q4:\n\n### Function: f3\n- **Code:**\n```\n@Q4 Borrower: f3()[a] (a == 550){\n    a -o Lender\n    a -> field4;\n    _\n} ==> @Q5\n```\n- **State of Invocation:** @Q4\n- **Pre-conditions for Invocation:** The asset `a` must be equal to 550.\n- **Operations Triggered:**\n  - The asset `a` is transferred to the Lender (`a -o Lender`).\n  - The value of `a` is assigned to `field4` (`a -> field4`).\n  - The contract transitions to state @Q5.\n\n### Function: fn (first occurrence)\n- **Code:**\n```\n@Q4 Lender: fn()[] (field3 > 0){\n    "notice" -> Borrower;\n    now + 48 >> @Q4 {"default" -> Borrower} ==> @Qd\n} ==> @Q4\n```\n- **State of Invocation:** @Q4\n- **Pre-conditions for Invocation:** The value of `field3` must be greater than 0.\n- **Operati

In [ ]:
clean_history("Cod_com2")
res = prompt("Cod_com2", f"{prompt_states_func}\n\n Contract code: {contract_loan_g}")
res1 = prompt("Cod_com2", f"{prompt_states_func2}\n\nfunctions:\n\n{res}")
print_response(res1)

{"deleted":true}
Connection opened!
Connection closed!
{'type': 'chat', 'user_id': 'Cod_com2', 'content': 'Based on the provided contract code, here are the functions that show the state @Q4 in their whole body:\n\n- f2: \n``` \n@Q2 Lender: f2()[l] (l == field1){\n    l -o Borrower;\n    t_1 >> @Q3 {"payment_due" -> Borrower} ==> @Q4\n    t_1a >> @Q4 {550 -> field3} ==> @Q4\n    t_2 >> @Q5 {"payment_due" -> Borrower} ==> @Q6\n    t_2a >> @Q6 {525 -> field3} ==> @Q6\n} ==> @Q3\n```\n\n- f3: \n```\n@Q4 Borrower: f3()[a] (a == 550){\n    a -o Lender\n    a -> field4;\n    _\n} ==> @Q5\n```\n\n- fn (first occurrence): \n```\n@Q4 Lender: fn()[] (field3 > 0){\n    "notice" -> Borrower;\n    now + 48 >> @Q4 {"default" -> Borrower} ==> @Qd\n} ==> @Q4\n```', 'why': {'input': '\nYou will receive a contract written in the stipula programming language.\n- identify all the functions that show the state @Q4 in their whole body;\n- return a list of all functions found and print the referenced piece o

In [ ]:
clean_history("ciao")
res10 = prompt("ciao", f"Contract:\n\n{contract_loan2_nat} \n\n{prompt_con1}\n\nfunctions:\n\n{fun10}")
print_response(res10)

{"deleted":true}
Connection opened!
Connection closed!
{'type': 'chat', 'user_id': 'ciao', 'content': '**Contract: Events of Default**  \n**Function: fn (first occurrence)**  \n**Function aspect: Pre-conditions for invocation: The value of `field3` must be greater than 0.**  \n**Explanation:** This aspect of the function corresponds to the contract\'s stipulation that an Event of Default occurs if the Borrower fails to make timely payment. The `field3` likely represents an amount due, and if it\'s greater than 0, it indicates a payment has not been made.  \n**Contract text:** "Borrower shall fail to make timely payment of any amount due to Lender hereunder;"\n\n**Function aspect: Operations triggered: A "notice" is sent to the Borrower.**  \n**Explanation:** This operation implements the contract\'s requirement for the Lender to notify the Borrower of an Event of Default. Sending a "notice" to the Borrower aligns with giving notice of the occurrence of an Event of Default.  \n**Contrac

In [ ]:
clean_history("ciao")
res10 = prompt("ciao", f"Contract:\n\n{contract_loan2_nat} \n\n{prompt_con1}\n\nfunctions:\n\n{fun10}")
print_response(res10)

{"deleted":true}
Connection opened!
Connection closed!
{'type': 'chat', 'user_id': 'ciao', 'content': '**Contract: Events of Default**  \n**Function: fn (first occurrence)**  \n**Function aspect: Pre-conditions for invocation: The value of `field3` must be greater than 0.**  \n**Explanation:** This aspect of the function corresponds to the contract\'s stipulation that an Event of Default occurs if the Borrower fails to make timely payment. The "value of `field3`" likely represents an amount due, and if it is greater than 0, it indicates that a payment has not been made as required.  \n**Contract text:** "Borrower shall fail to make timely payment of any amount due to Lender hereunder;"\n\n**Function aspect: Operations triggered: A "notice" is sent to the Borrower.**  \n**Explanation:** This operation implements the contract\'s requirement for the Lender to notify the Borrower of an Event of Default. Sending a "notice" to the Borrower aligns with the contract\'s provision for giving not

In [ ]:
clean_history("ciao")
res10 = prompt("ciao", f"Contract:\n\n{contract_loan2_nat} \n\n{prompt_con1}\n\nfunctions:\n\n{fun10}")
print_response(res10)

{"deleted":true}
Connection opened!
Connection closed!
{'type': 'chat', 'user_id': 'ciao', 'content': '**contract:** Repayment  \n**function:** f3  \n**function aspect:** The asset `a` must be equal to 550.  \n**explanation:** This corresponds to the first payment due under the contract, where the Borrower is required to pay $550 on June 1, 2023. The function\'s precondition that `a` must equal 550 directly represents this payment amount.  \n**contract text:** "Payment 1, due June 1, 2023, in the amount of $550, representing a payment of $500 as half of the principal and interest in the amount of $50."\n\n**function aspect:** The asset `a` is transferred to the Lender using the `-o` operator.  \n**explanation:** This operation represents the actual payment from the Borrower to the Lender, fulfilling the repayment obligation specified in the contract for the first payment. The use of a unique or consuming transfer (`-o` operator) signifies that the payment amount is deducted from the Bo